MARIO JIMÉNEZ, ALEJANDRO RAMÍREZ Y DAVID SEIJAS

# ENUNCIADO

Un grupo de 5 personas quiere cruzar un viejo y estrecho puente. Es una noche
cerrada y se necesita llevar una linterna para cruzar. El grupo solo dispone de
una linterna, a la que le quedan 5 minutos de batería.
    1. Cada persona tarda en cruzar 10, 30, 60, 80 y 120 segundos,
        respectivamente.
    2. El puente solo resiste un máximo de 2 personas cruzando a la vez, y
        cuando cruzan dos personas juntas, caminan a la velocidad del más lento.
    3. No se puede lanzar la linterna de un extremo a otro del puente, así que
        cada vez que crucen dos personas, alguien tiene que volver a cruzar hacia
        atrás con la linterna a buscar a los compañeros que falten, y así hasta que
        hayan cruzado todos.

# RESOLUCION

La entrada consiste en el número de personas, así como sus respectivos tiempos de cruce y la duración de la linterna.
El estado va a ser un array de tamaño k="personas" + 2;
    1. Las primeras "personas" posiciones se corresponden a la orilla en la que se encuentra la persona (0 es que se encuentra en la orilla inicio y 1 en la orilla final).
    2. La siguiente posición (state[personas]) es la orilla en la que está la linterna.
    3. La última posición se corresponde que el tiempo que le queda a la linterna para que se apague. Obviamente la solución será válida si han cruzado todos y dicho tiempo es >= 0

# REPRESENTACIÓN DE ESTADOS

Llevamos en el estado una tupla (P1, P2,..., Pn, L, t) donde
- Pi es 0 si la persona i está en la orilla origen o 1 si está en la orilla destino (i entre 1 y n)
- n el número de personas (variable dependiendo del problema)
- L 0 o 1 dependiendo de la orilla en la que esté la linterna
- t el tiempo que resta a la linterna con t < tLimite (se pasará por parámetro, igual que n, al crear el problema)

---

### Estado inicial
(0, 0, 0, 0, 0, 0, 300) suponiendo nuestro problema con 5 personas y tiempoLimite 300

### Estado objetivo 
(1, 1, 1, 1, 1, 1, t) con t en [0,300]

---

#### Estados posibles
2^(n+1)

#### Estados inalcanzables 
(0, 0, 0, 0, 0, 1, t)
(1, 1, 1, 1, 1, 0, t)
con t entre 0 y tLimite=300

#### Restricciones estados no válido (estados de peligro):
t < 0.
Todos los estados alcanzables son válidos pues no hay ningún peligro en el problema, solo pondremos que el tiempo tiene que ser < 0, es decir, la linterna todavía tiene batería

---

### Especificación operadores
cruzaij(i,j)
donde cruzarán a la otra orilla las personas i y j, o la persona i sola si j = 0 (en el código jugamos con los índices de los estados pues la persona 1 en la lista del estado estaría en la posición 0)

PRECONDICIONES:
(L=0 and Pi = 0 and *Pj = 0*) or (L=1 and Pi = 1 and *Pj = 1*) 

POSTCONDICIONES: 

- (P1,...,Pi,...,Pj,..., L, t) -> (P1,...,Pi + 1,...,Pj *+ 1*,..., L + 1, t)
- (P1,...,Pi,...,Pj,..., L, t) -> (P1,...,Pi - 1,...,Pj *- 1*,..., L - 1, t)
- Estado no peligroso (t' >= 0)

*Lo que está en cursiva ocurre solo si j !=0, si no se omite

### Coste de la solución
EL coste será el número de operadores aplicados si utilizamos como heurística el número de viajes que nos quedan para llegar todos a la orilla destino y lleguemos al estado objetivo. Si utilizamos heurísticas en las que influye el tiempo cambiamos el coste al implementado en la clase del problema de este archivo que será el tiempo que llevamos con los viajes realizados hasta el momento

In [2]:
 cd ..

C:\Users\david\Desktop\Trabajos\4º\IA\Prácticas\PR1


In [3]:
cd ..


C:\Users\david\Desktop\Trabajos\4º\IA\Prácticas


In [4]:
cd aima-python


C:\Users\david\Desktop\Trabajos\4º\IA\Prácticas\aima-python


In [5]:
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

In [6]:
import math

class Problema1 (Problem):
    
    
    def __init__(self, personas, tiempos, tLimite, goal=None):
        self.tiempos = tiempos
        self.personas = personas
        
        initial = [0 for n in range (0,personas+1)]
        initial.append(tLimite)
        self.initial = tuple(initial)
        
        goal=[1 for n in range (0,personas+1)]
        goal.append(0)
        self.goal = tuple(goal)
        
        self._actions = [("Cruza la persona %d"%i,(i,0)) for i in range (1,personas + 1)]
        self._actions += [("Cruzan las personas %d y %d"%(i,j),(i,j)) for i in range (1,personas + 1) for j in range (i+1,personas + 1)]
        
        
    def actions(self, s):
        valido = True
        lista = []
        
        for a in self._actions:
            if a[1][1] == 0: #Un 0 en la segunda poscion de la tupla que indica que dos personas cruzan significa que sólo cruza una
                valido = s[a[1][0] - 1] == s[self.personas]#para comprobar que el que se mueve esta en la misma orilla que la linterna
                valido = valido and (s[self.personas + 1] - self.tiempos[a[1][0] - 1])>=0 #para comprobar que todavia tenemos tiempo para que puedan cruzar la personas
            else:
                valido = s[a[1][0] - 1] == s[a[1][1] - 1] == s[self.personas]
                valido = valido and (s[self.personas + 1] - max(self.tiempos[a[1][0] - 1],self.tiempos[a[1][1] - 1]))>=0
                
            if valido:
                lista.append(a)
                
        return lista
    '''
    Cada vez que se mete en is valid modifica el estado actual, por lo que las comprobaciones despues de la primera vez son erroneas
     def _is_valid(self, s):
        valido = True
        for i in range (0, self.personas + 1):
            if not(0 <= s[i] <= 1):
                valido = False
                
        if s[self.personas + 1]<0:
            valido = False
        
        return valido
    
    '''
   
    
    def result(self, s, a):
        l = list(s)
        maximo = self.tiempos[a[1][0]-1]
        if l[self.personas] == 0:
            l[a[1][0] - 1] += 1 #actualiza el que se mueve
            l[self.personas] += 1 #actualiza la linterna
            
            if a[1][1] != 0:
                maximo = max(maximo,self.tiempos[a[1][1]-1])
                l[a[1][1] - 1] += 1
            
        else:
            l[self.personas] -= 1
            l[a[1][0] - 1] -= 1
            
            if a[1][1] != 0:
                maximo = max(maximo,self.tiempos[a[1][1]-1])
                l[a[1][1] - 1] -= 1
        
        l[self.personas + 1] -= maximo #actualiza el tiempo
                
        return tuple(l)
    
    def goal_test(self, state):
        i = 0
        while i < self.personas+1:
            if state[i] != self.goal[i]:
                return False
            i += 1
            
        return state[i]>=0
    
    
    def path_cost(self, c, state1, action, state2):   #Para las heurísticas en las que miramos el tiempo
        cost = self.tiempos[action[1][0] - 1]
        if action[1][1] != 0:
            cost = max(self.tiempos[action[1][1] - 1], cost)
    
        return c + cost
    

    #HEURÍSTICAS
   
    
    
        
    def h(self, node):                  #hTiempoMin
        state = node.state
        viajes = 0
        minTiempo = self.tiempos[0]
            
        for i in range (0,self.personas):
            if state[i] == 0:
                viajes += 1
                
        if viajes != 0:
            viajes += state[self.personas] #si la linterna esta en el destino, tiene que cruzar alguien con ella y entonces habrá una persona más en el origen que tenga que cruzar
            
        if viajes >= 2:
            viajes = (viajes-2)*2 + 1 + state[self.personas] #todas las personas menos los dos ultimos hacen dos viajes para que 
        

        
        for i in range (1, self.personas):
            minTiempo = min(minTiempo, self.tiempos[i])
            
        return viajes*minTiempo
                         
           
    def hViajes(self, node):                     #hViajes
        state = node.state
        num = 0
        for i in range (0,self.personas):
            if state[i] == 0:
                num += 1
                
        if num != 0:
            num += state[self.personas] #si la linterna esta en el destino, tiene que cruzar alguien con ella y entonces habrá una persona más en el origen que tenga que cruzar
            
        if num < 2:
            return num
        else:
            return (num-2)*2 + 1 + state[self.personas] #todas las personas menos los dos ultimos hacen dos viajes para que cruce uno solo y los dos últimos vuelven en un vijae. Además si la linterna empezaba en la orilla destino tiene que ir a la orilla origen en un viaje     
    
    
    def hInfLinternas(self, node):               #hInfLinternas
        state = node.state
        tiempo = 0
        
        for i in range (0,self.personas):
            if state[i] == 0:
                tiempo += self.tiempos[i]
        
        return tiempo/2
                  

### EXPLICACIÓN HERÍSTICAS
Definimos tres heurísticas:
- hViajes: cálcula la cantidad de viajes mínimo que tenemos que hacer para llegar al estado objetivo según las personas que nos quedan en el lado origen. Cuando utilicemos esta el coste de cada operdor será uno (habría que comentar la función coste sobrescrita en la clase de arriba)
- hTiempo: cálcula el tiempo mínimos que queda para llegar al estado objetivo según el número de personas que quedan por cruzar. Coge el número de viajes mínimo que habría que hacer y lo multiplica por el tiempo de la persona que menos tarda en cruzar. Aquí el coste de los operadores sería el tiempo que tarda en cruzar el puente en cada viaje
- hInfLinternas: es la menos informada de todas. Suponemos que tenemos infinitas linternas y que hay que hacer viajes solo del lado origen al lado destino

In [7]:
p=Problema1(5,[10,30,60,80,120],5*60)

In [8]:
%%timeit
breadth_first_graph_search(p).solution()

139 ms ± 7.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%%timeit
depth_first_graph_search(p).solution()

30.8 ms ± 5.11 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%%timeit
uniform_cost_search(p).solution()

805 ms ± 253 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
best_first_graph_search(p, p.h).solution()

94.8 ms ± 2.98 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%%timeit
best_first_graph_search(p, p.hViajes).solution()

82.2 ms ± 14.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%%timeit
best_first_graph_search(p, p.hInfLinternas).solution()

92 ms ± 5.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
best_first_graph_search(p, p.hInfLinternas).solution()

[('Cruzan las personas 1 y 3', (1, 3)),
 ('Cruza la persona 1', (1, 0)),
 ('Cruzan las personas 1 y 2', (1, 2)),
 ('Cruza la persona 1', (1, 0)),
 ('Cruzan las personas 4 y 5', (4, 5)),
 ('Cruza la persona 2', (2, 0)),
 ('Cruzan las personas 1 y 2', (1, 2))]

In [15]:
%%timeit
astar_search(p).solution()

695 ms ± 130 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%%timeit
astar_search(p, p.hViajes).solution()

864 ms ± 72 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit
astar_search(p, p.hInfLinternas).solution()

408 ms ± 27.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit
astar_search(p).solution()

699 ms ± 70.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
breadth_first_graph_search(p).solution()

[('Cruzan las personas 1 y 2', (1, 2)),
 ('Cruza la persona 1', (1, 0)),
 ('Cruzan las personas 1 y 3', (1, 3)),
 ('Cruza la persona 1', (1, 0)),
 ('Cruzan las personas 4 y 5', (4, 5)),
 ('Cruza la persona 2', (2, 0)),
 ('Cruzan las personas 1 y 2', (1, 2))]

In [20]:
depth_first_graph_search(p).solution()

[('Cruzan las personas 1 y 3', (1, 3)),
 ('Cruza la persona 1', (1, 0)),
 ('Cruzan las personas 1 y 2', (1, 2)),
 ('Cruza la persona 2', (2, 0)),
 ('Cruzan las personas 4 y 5', (4, 5)),
 ('Cruza la persona 1', (1, 0)),
 ('Cruzan las personas 1 y 2', (1, 2))]

In [21]:
uniform_cost_search(p).solution()

[('Cruzan las personas 1 y 2', (1, 2)),
 ('Cruza la persona 1', (1, 0)),
 ('Cruzan las personas 4 y 5', (4, 5)),
 ('Cruza la persona 2', (2, 0)),
 ('Cruzan las personas 1 y 2', (1, 2)),
 ('Cruza la persona 1', (1, 0)),
 ('Cruzan las personas 1 y 3', (1, 3))]

In [22]:
astar_search(p).solution()

[('Cruzan las personas 1 y 2', (1, 2)),
 ('Cruza la persona 1', (1, 0)),
 ('Cruzan las personas 4 y 5', (4, 5)),
 ('Cruza la persona 2', (2, 0)),
 ('Cruzan las personas 1 y 2', (1, 2)),
 ('Cruza la persona 1', (1, 0)),
 ('Cruzan las personas 1 y 3', (1, 3))]

In [23]:
best_first_graph_search(p, p.h).solution()

[('Cruzan las personas 1 y 3', (1, 3)),
 ('Cruza la persona 1', (1, 0)),
 ('Cruzan las personas 1 y 2', (1, 2)),
 ('Cruza la persona 1', (1, 0)),
 ('Cruzan las personas 4 y 5', (4, 5)),
 ('Cruza la persona 2', (2, 0)),
 ('Cruzan las personas 1 y 2', (1, 2))]

Las cinco búsquedas son instántaneas

In [24]:
# Hacemos una definición ampliada de la clase Problem de AIMA que nos va a permitir experimentar con distintos
# estados iniciales, algoritmos y heurísticas, para resolver el 8-puzzle. 
# Añadimos en la clase ampliada la capacidad para contar el número de nodos analizados durante la búsqueda:


class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)       

In [25]:
def resuelve_p(problem, algoritmo, h=None):

    p=Problema_con_Analizados(problem)
    if h: 
        sol= algoritmo(p,h).solution()
    else: 
        sol= algoritmo(p).solution()
    print("Solución: {0}".format(sol))
    print("Algoritmo: {0}".format(algoritmo.__name__))
    if h: 
        print("Heurística: {0}".format(h.__name__))
    else:
        pass
    print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p.analizados))
  

In [26]:
resuelve_p(p, astar_search, p.h)

Solución: [('Cruzan las personas 1 y 3', (1, 3)), ('Cruza la persona 1', (1, 0)), ('Cruzan las personas 1 y 2', (1, 2)), ('Cruza la persona 1', (1, 0)), ('Cruzan las personas 4 y 5', (4, 5)), ('Cruza la persona 2', (2, 0)), ('Cruzan las personas 1 y 2', (1, 2))]
Algoritmo: astar_search
Heurística: h
Longitud de la solución: 7. Nodos analizados: 395


In [27]:
resuelve_p(p, astar_search, p.hViajes)

Solución: [('Cruzan las personas 1 y 2', (1, 2)), ('Cruza la persona 1', (1, 0)), ('Cruzan las personas 4 y 5', (4, 5)), ('Cruza la persona 2', (2, 0)), ('Cruzan las personas 1 y 3', (1, 3)), ('Cruza la persona 1', (1, 0)), ('Cruzan las personas 1 y 2', (1, 2))]
Algoritmo: astar_search
Heurística: hViajes
Longitud de la solución: 7. Nodos analizados: 250


In [28]:
resuelve_p(p, astar_search, p.hInfLinternas)

Solución: [('Cruzan las personas 1 y 3', (1, 3)), ('Cruza la persona 1', (1, 0)), ('Cruzan las personas 1 y 2', (1, 2)), ('Cruza la persona 1', (1, 0)), ('Cruzan las personas 4 y 5', (4, 5)), ('Cruza la persona 2', (2, 0)), ('Cruzan las personas 1 y 2', (1, 2))]
Algoritmo: astar_search
Heurística: hInfLinternas
Longitud de la solución: 7. Nodos analizados: 457


In [29]:
resuelve_p(p, best_first_graph_search, p.h)

Solución: [('Cruzan las personas 1 y 3', (1, 3)), ('Cruza la persona 1', (1, 0)), ('Cruzan las personas 1 y 2', (1, 2)), ('Cruza la persona 1', (1, 0)), ('Cruzan las personas 4 y 5', (4, 5)), ('Cruza la persona 2', (2, 0)), ('Cruzan las personas 1 y 2', (1, 2))]
Algoritmo: best_first_graph_search
Heurística: h
Longitud de la solución: 7. Nodos analizados: 536


In [30]:
resuelve_p(p, best_first_graph_search, p.hViajes)

Solución: [('Cruzan las personas 1 y 3', (1, 3)), ('Cruza la persona 1', (1, 0)), ('Cruzan las personas 1 y 2', (1, 2)), ('Cruza la persona 1', (1, 0)), ('Cruzan las personas 4 y 5', (4, 5)), ('Cruza la persona 2', (2, 0)), ('Cruzan las personas 1 y 2', (1, 2))]
Algoritmo: best_first_graph_search
Heurística: hViajes
Longitud de la solución: 7. Nodos analizados: 536


In [31]:
resuelve_p(p, best_first_graph_search, p.hInfLinternas)

Solución: [('Cruzan las personas 1 y 3', (1, 3)), ('Cruza la persona 1', (1, 0)), ('Cruzan las personas 1 y 2', (1, 2)), ('Cruza la persona 1', (1, 0)), ('Cruzan las personas 4 y 5', (4, 5)), ('Cruza la persona 2', (2, 0)), ('Cruzan las personas 1 y 2', (1, 2))]
Algoritmo: best_first_graph_search
Heurística: hInfLinternas
Longitud de la solución: 7. Nodos analizados: 502


In [32]:
resuelve_p(p, breadth_first_graph_search)

Solución: [('Cruzan las personas 1 y 2', (1, 2)), ('Cruza la persona 1', (1, 0)), ('Cruzan las personas 1 y 3', (1, 3)), ('Cruza la persona 1', (1, 0)), ('Cruzan las personas 4 y 5', (4, 5)), ('Cruza la persona 2', (2, 0)), ('Cruzan las personas 1 y 2', (1, 2))]
Algoritmo: breadth_first_graph_search
Longitud de la solución: 7. Nodos analizados: 720


In [33]:
resuelve_p(p, depth_first_graph_search)

Solución: [('Cruzan las personas 1 y 3', (1, 3)), ('Cruza la persona 1', (1, 0)), ('Cruzan las personas 1 y 2', (1, 2)), ('Cruza la persona 2', (2, 0)), ('Cruzan las personas 4 y 5', (4, 5)), ('Cruza la persona 1', (1, 0)), ('Cruzan las personas 1 y 2', (1, 2))]
Algoritmo: depth_first_graph_search
Longitud de la solución: 7. Nodos analizados: 610


In [34]:
resuelve_p(p, uniform_cost_search)

Solución: [('Cruzan las personas 1 y 3', (1, 3)), ('Cruza la persona 1', (1, 0)), ('Cruzan las personas 1 y 2', (1, 2)), ('Cruza la persona 1', (1, 0)), ('Cruzan las personas 4 y 5', (4, 5)), ('Cruza la persona 2', (2, 0)), ('Cruzan las personas 1 y 2', (1, 2))]
Algoritmo: uniform_cost_search
Longitud de la solución: 7. Nodos analizados: 720


### COMENTARIOS DE HEURÍSTICAS Y BÚSQUEDAS
    1. En primer lugar, como hemos analizado en las respectivas versiones con la clase Problema_con_Analizados la mejor solución viene de usar búsqueda informada o heurística. En concreto, tenemos que analizar menos nodos con A* y de todas las heurísticas propuestas la mejor es hViajes, con 250, en 2º lugar hTiempoMin (h)(395) y en el tercer lugar hInfLinternas(497). 

    2. hViajes es más informada que hTiempoMin pues hViajes mira el número mínimo de pasos que nos quedaría siendo el coste de cada viaje 1, sin embargo, en hTiempoMin (coste del viaje = el tiempo que se tarda) multiplica esos viajes mínimos por el menor de los tiempos de todas las personas (no por el menor tiempo de los viajes posibles).

    3. En la última heurística la información que manejamos es muy escasa y la heurística es demasiado 'optimista', por eso es la menos informada. 

    4. Por otro lado, entre A* y la búsqueda voraz es preferible la primera pues tenemos en cuenta el coste del camino que llevamos, siendo en este caso bastante importante saber cuánto tiempo (o viajes) llevamos por la duración de la batería de la linterna. 

    5. Vemos que las búsquedas ciegas no encajan con este problema pues es importante llevar una información sobre lo prometedores que son los caminos posibles pues tenemos que terminar el problema con el menor tiempo de linterna utilizado. Por lo tanto, estas heurísticas analizan más nodos de los necesarios.

#### Factor ramificación:
Al inicio hay 5 personas a la izquierda y tenemos 15 posibles acciones (cruce de las 5 personas solas y las combinaciones de las 5 cogiendo 2). Después tendríamos dos casos: que cruce una sola persona y tenga que volver y tendríamos la situación inicial (no tiene sentido) o que crucen dos y vuelva una y nos quedarían 4 personas en la orilla origen y tendríamos 10 posibles acciones. Por lo tanto, según vamos aumentando la profundidad se va reduciendo el factor de ramificación para profundidad para y va aumentando para profundidad impar.